In [36]:
import requests, json
from dataclasses import dataclass, asdict

coins = { "BTC", "BCH", "DOGE", "LTC", "KAS", "RVN"}
api_token = "8a0b92957b57d4eea9cdf73c4d7436c09c82a38865b84e5b2304b7b5a0202b92"
coins_endpoint = f"https://whattomine.com/api/v1/coins?api_token={api_token}"
json = requests.get(coins_endpoint).json()

@dataclass
class WhatToMineData:
    tag: str = ""
    block_time: float = 0
    block_reward_btc: float = 0
    difficulty: float = 0
    nethash: float = 0
    daily_emissions_btc: float = 0
    algorithm: str = ""

def toWhatToMineData(json):
    data = WhatToMineData()
    data.tag = json['tag']
    data.block_time = float(json['block_time'])
    exchange_rate_btc = 1 if data.tag == "BTC" else json['exchanges'][0]['price']
    data.block_reward_btc = json['block_reward'] * exchange_rate_btc
    data.difficulty = json['difficulty']
    data.nethash = json['nethash']
    data.daily_emissions_btc = 86400 / data.block_time * data.block_reward_btc
    data.algorithm = json['algorithm']
    return data

coins_data = [ toWhatToMineData(coin) for coin in json if coin['tag'] in coins ]
print("Global coins data:\n")
for coin_data in coins_data:
    print(asdict(coin_data))


Global coins data:

{'tag': 'BCH', 'block_time': 630.0, 'block_reward_btc': 0.010710572654957309, 'difficulty': 372691227035.65, 'nethash': 2540788304165441269, 'daily_emissions_btc': 1.4688785355370022, 'algorithm': 'SHA-256'}
{'tag': 'BTC', 'block_time': 651.0, 'block_reward_btc': 3.1561542979999997, 'difficulty': 108105433845147.0, 'nethash': 713224735614128878648, 'daily_emissions_btc': 418.8813077529954, 'algorithm': 'SHA-256'}
{'tag': 'DOGE', 'block_time': 63.0, 'block_reward_btc': 0.02718, 'difficulty': 39457219.81731997, 'nethash': 2689959820737655, 'daily_emissions_btc': 37.27542857142857, 'algorithm': 'Scrypt'}
{'tag': 'KAS', 'block_time': 1.0004, 'block_reward_btc': 6.602872024910351e-05, 'difficulty': 313136175.4059518, 'nethash': 1344371883809558176, 'daily_emissions_btc': 5.702600389366796, 'algorithm': 'kHeavyHash'}
{'tag': 'LTC', 'block_time': 150.0, 'block_reward_btc': 0.006738125, 'difficulty': 74389147.7639597, 'nethash': 2129993045490122, 'daily_emissions_btc': 3.88

In [37]:
import requests, json
import pandas as pd

algorithms = { "SCRYPT", "SHA256ASICBOOST", "KHEAVYHASH", "KAWPOW"}

multialgo_response = requests.get('https://api2.nicehash.com/main/api/v2/public/simplemultialgo/info')
multialgo_data = multialgo_response.json()
algo_data = { element['algorithm']: {'paying': element['paying'], 'speed': element['speed']} for element in multialgo_data['miningAlgorithms'] if element['algorithm'] in algorithms }

algorithms_response = requests.get('https://api2.nicehash.com/main/api/v2/mining/algorithms')
algorithms_response.raise_for_status()
algorithms_data = algorithms_response.json()

index = [ element['algorithm'] for element in algorithms_data['miningAlgorithms'] if element['algorithm'] in algorithms ]
columns = ['marketFactor', 'displayMarketFactor', 'priceFactor', 'displayPriceFactor', 'miningFactor', 'displayMiningFactor', 'priceDownStep', 'paying', 'speed']
data = [ [ float(element['marketFactor']), element['displayMarketFactor'], float(element['priceFactor']), element['displayPriceFactor'], float(element['miningFactor']), element['displayMiningFactor'], float(element['priceDownStep']), algo_data[element['algorithm']]['paying'], algo_data[element['algorithm']]['speed'] ] for element in algorithms_data['miningAlgorithms'] if element['algorithm'] in algorithms ]

raw_data_df = pd.DataFrame(data, index=index, columns=columns)

calc_df = pd.DataFrame(index=index)
sats2btc = 1e8

calc_df['hashrate'] = raw_data_df['speed'] / raw_data_df['marketFactor'] * raw_data_df['miningFactor']
calc_df['hashrate unit'] = raw_data_df['displayMarketFactor'] + '/s'

calc_df['paying'] = raw_data_df['paying'] * raw_data_df['priceFactor'] / sats2btc
calc_df['paying unit'] = 'BTC/' + raw_data_df['displayPriceFactor'] + '/day'

calc_df['daily_consumption_capacity'] = calc_df['hashrate'] * calc_df['paying']
calc_df['daily_consumption_capacity unit'] = 'BTC/day'

print("Nicehash data:\n")
json.loads(calc_df.to_json(orient='index'))


Nicehash data:



{'SCRYPT': {'hashrate': 112.7946190438,
  'hashrate unit': 'TH/s',
  'paying': 0.0168254991,
  'paying unit': 'BTC/TH/day',
  'daily_consumption_capacity': 1.8978257653,
  'daily_consumption_capacity unit': 'BTC/day'},
 'SHA256ASICBOOST': {'hashrate': 6.6853179358,
  'hashrate unit': 'EH/s',
  'paying': 0.569080555,
  'paying unit': 'BTC/EH/day',
  'daily_consumption_capacity': 3.804484441,
  'daily_consumption_capacity unit': 'BTC/day'},
 'KAWPOW': {'hashrate': 0.8824737322,
  'hashrate unit': 'TH/s',
  'paying': 0.0841279322,
  'paying unit': 'BTC/TH/day',
  'daily_consumption_capacity': 0.0742406903,
  'daily_consumption_capacity unit': 'BTC/day'},
 'KHEAVYHASH': {'hashrate': 54.2565007842,
  'hashrate unit': 'PH/s',
  'paying': 0.0041252307,
  'paying unit': 'BTC/PH/day',
  'daily_consumption_capacity': 0.2238205807,
  'daily_consumption_capacity unit': 'BTC/day'}}